# Data Cleaning
city names source: https://simplemaps.com/data/us-cities

In [94]:
import os
import json
import time
import requests
import datetime
import dateutil
import pandas as pd
import glob
import string
from dateutil.relativedelta import relativedelta

In [80]:
names = pd.read_csv("./Data/names.csv", encoding = "ISO-8859-1")
locations = pd.read_csv("./Data/uscities.csv")
articles = pd.read_csv("./headlines/NYTData.csv")
countries = pd.read_csv("./Data/countries.csv")

In [209]:
# Formatting locations and creating a locationSet
locationSet = {"United States"}
# countries = countries.drop(columns=["Code"])
for country in countries.Name.values:
    if len(country.split()) > 1:
        locationSet.add(country.lower()) 
for location in locations.city.values:
    if len(location.split()) > 1:
        locationSet.add(location.lower()) 
for location in locations.state_name.values:
    if len(location.split()) > 1:
        locationSet.add(location.lower()) 

In [81]:
articles = articles.drop(columns=["doc_type", "material_type", "section", "keywords"])
articles = articles.dropna()
names = names.drop(columns=["A"])

In [82]:
names.columns = ["names"]
names = names.append(pd.DataFrame(["Adele"], columns=["names"]))
# How to check is a word is in the dataframe
#'Adele' in names.names.values
names["names"][4092] = "Adele"
names["names"][0] = "Alf"

In [149]:
#Removing punctuationg from headlines
for i in range(len(articles)):
    articles.iloc[i][0] = articles.iloc[i][0].translate(str.maketrans('', '', string.punctuation)).lower()

KeyError: 'Date'

In [152]:
#Group by every day
articles = articles.groupby(["date"]).agg(lambda s: " ".join(s))

In [220]:
#Formatting names
i = 0
nameSet = {"TESTSTRING"}
nameSet.remove("TESTSTRING")
for name in names.names.values:
    names["names"][i] = name.lower()
    i += 1
    nameSet.add(name)

In [ ]:
#Testing below
# d = {'col1': [1, 2], 'headline': ["hi hi hi hi hi hi", "potato potato potato hi hi hi hi"]}
# testdf = pd.DataFrame(data=d)
# for i in range(len(testdf)):
#     s = {}
#     for name in nameSet:
#         if name in testdf["headline"][i]:
#             if name in s:
#                 s[name] += 1
#             else:
#                 s[name] = 1
#             testdf["headline"][i].replace(name, "")
#     for name in locationSet:
#         if name in testdf["headline"][i]:
#             if name in s:
#                 s[name] += 1
#             else:
#                 s[name] = 1
#             testdf["headline"][i].replace(name, "")
#     for word in testdf["headline"][i].split():
#         if word in s:
#             s[word] += 1
#         else:
#             s[word] = 1
#     testdf["headline"][i] = s

In [229]:
#Removing duplicate words from headlines and counting them
for i in range(len(articles)):
    s = {}
    for name in nameSet:
        if name in articles["headline"][i]:
            if name in s:
                s[name] += 1
            else:
                s[name] = 1
            articles["headline"][i].replace(name, "")
    for name in locationSet:
        if name in articles["headline"][i]:
            if name in s:
                s[name] += 1
            else:
                s[name] = 1
            articles["headline"][i].replace(name, "")
    for word in articles["headline"][i].split():
        if word in s:
            s[word] += 1
        else:
            s[word] = 1
    articles["headline"][i] = s
articles.to_csv("NYTDataCleaned.csv", encoding = "ISO-8859-1")

<ipython-input-229-422b30b678ec>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testdf["headline"][i] = s


,col1,headline
0,1,{'hi': 6}
1,2,"{'potato': 3, 'hi': 4}"
